In [17]:
import numpy as np
import pandas as pd
import glob

import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from pandas.plotting import scatter_matrix

from os import listdir
from os.path import isfile, join

import math
from scipy.special import gamma

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from IPython.display import display
import json

from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics, svm

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('loan_data.csv', index_col=0)

/Users/hzaman/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# estimating default rate
df_v1 = df.loc[df['loan_status'].isin(['Default', 'Fully Paid','Charged Off'])]
# lumping both default and charged off under the same category of default and turning variables into categorical
df_v1['loan_status'] = df_v1['loan_status'].map({'Default': 1, 'Charged Off': 1, 'Fully Paid': 0.0})

/Users/hzaman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
# defining categorical and continuous variables
cat_columns = ['loan_status', 'sub_grade', 'home_ownership','purpose', 'grade', 'mort_acc', 'emp_length', 'addr_state'] 
cts_columns = ['loan_amnt','int_rate','dti', 'inq_last_6mths', 'open_acc', 'total_acc', 'revol_util', 'total_pymnt',
              'revol_bal', 'annual_inc']
cat_cts_columns = cat_columns + cts_columns

In [5]:
# Making a function to handle categorical variables. While the following function assigns numerical values
def make_cat_dict(arr):
    a = list(set(arr.tolist()))
    d = {}
    for i in range(0,len(a)):
        d[a[i]] = i
    return d



In [6]:
# Making copies of the dataframe with categorical and cts variables
df_rf_cat = df_v1[cat_columns]
df_rf_cts = df_v1[cts_columns]
df_rf = df_v1[cat_cts_columns]

# removing % sign from the data whereever it exits
df_rf['int_rate'] = df_rf['int_rate'].str.rstrip('%').astype('float')
df_rf['revol_util'] = df_rf['revol_util'].str.rstrip('%').astype('float')
my_test_df = df_rf.copy()

cat_val_dict = {}
# Encoding the categorical variables
for category in cat_columns:
    print(category + " is now recoded!")
    # df[category] = df[category].astype('category')
    cat_dict = make_cat_dict(df_rf[category])
    df_rf[category] = df_rf[category].map(cat_dict)
    cat_val_dict[category] = cat_dict
    # df_rf[category] = pd.Categorical.from_array(df_rf_cat[category]).labels
    

# Accessing the head of the dataframe
df_rf.head()

/Users/hzaman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/hzaman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


loan_status is now recoded!


/Users/hzaman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


sub_grade is now recoded!
home_ownership is now recoded!
purpose is now recoded!
grade is now recoded!
mort_acc is now recoded!
emp_length is now recoded!
addr_state is now recoded!


,loan_status,sub_grade,home_ownership,purpose,grade,mort_acc,emp_length,addr_state,loan_amnt,int_rate,dti,inq_last_6mths,open_acc,total_acc,revol_util,total_pymnt,revol_bal,annual_inc
43,0,13,2,6,3,2,7,46,6000.0,22.90,32.77,0.0,10.0,29.0,85.9,6281.441670,36170.0,74000.0
159,0,14,2,7,5,0,6,6,1000.0,20.39,12.75,0.0,2.0,2.0,60.5,1042.772353,3627.0,9600.0
259,0,13,5,6,3,7,7,6,5000.0,22.90,15.57,1.0,8.0,18.0,64.6,5201.191962,12407.0,43700.0
271,0,17,1,6,3,0,6,5,12000.0,26.77,16.13,1.0,9.0,11.0,60.7,12575.226488,7589.0,54000.0
280,0,21,1,10,1,0,10,14,5000.0,13.06,5.33,0.0,14.0,19.0,42.1,5049.002132,3913.0,35360.0


In [7]:
# Estimating probability of default given loan characteristics
# Splitting the dataset
df_v2 = df_rf.copy()

y = df_v2['loan_status']
x = df_v2.drop(['loan_status'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=99)

In [8]:
# Fitting a random forest
n_trees, maxf = 45, 14

est = RandomForestClassifier(oob_score=True, n_estimators = n_trees, max_depth=20)
rf1 = est.fit(X_train, y_train)
print("OOB, accuracy:", rf1.oob_score_)

train_acc_rf1 = rf1.score(X_train, y_train)
print("The accuracy on training set:",train_acc_rf1)

test_acc_rf1 = rf1.score(X_test, y_test)
print("The accuracy on test set:", test_acc_rf1)

OOB, accuracy: 0.9811493289389711
The accuracy on training set: 0.992479734814208
The accuracy on test set: 0.9820955138768499


In [9]:
# Predicting probability of default
print(est.predict_proba(X_train))
# print(est.predict_proba(X_test))

# Assuming the target is (0,1) => (full_paid, default/charged-off), the pair of probabilities we get below
# 

[[0.94814037 0.05185963]
 [0.99314436 0.00685564]
 [0.98020915 0.01979085]
 ...
 [0.97667333 0.02332667]
 [0.96204923 0.03795077]
 [0.98667422 0.01332578]]


In [10]:
# esimated_model is the model estimated using tarining data
# new_entry corresponds to a new loan that we are going to calculate default probability on
# cat_cols is the list of all categorical columns
# cat_val_dict is the dictionary of dictionaries that includes a categoerical variables and their 
# underlying entries from training data

def predict_default_prob(estimated_model, new_entry, cat_cols, cat_val_dict):
    #print(new_entry)
    # first for loop takes categorical columns of new_entry as input and updates their values 
    # according to the values in cat_val_dict
    # cat_val_dict includes all the parameter values from the training model
    for col in cat_cols:
        if(col != 'loan_status'):
            d = cat_val_dict[col]
            new_entry[col] = d[new_entry[col]]

    arr = np.array(list(new_entry.values())).reshape(1,len(new_entry))
    # now estimating the probability of default for a new loan
    prob = estimated_model.predict_proba(arr)
    return prob[0][1]  # [0][1] gives probability of default. [0][0] gives probability of no deafult

In [11]:
# Let's test a new loan to see if our model correctly predicts default probability
data = my_test_df.copy()
y_var = data['loan_status']
x_var = data[cat_cts_columns]
x_var = x_var.drop(['loan_status'], axis = 1)

row = x_var.iloc[0].to_dict()
# sample_row = {'sub_grade': 'E1', 'home_ownership': 'OWN', 'purpose': 'debt_consolidation', 'grade': 'E', 
#               'mort_acc': 2.0, 'emp_length': '6 years', 'addr_state': 'GA', 'loan_amnt': 6000.0, 
#               'int_rate': 22.9, 'dti': 32.77, 'inq_last_6mths': 0.0, 'open_acc': 10.0, 
#               'total_acc': 29.0, 'revol_util': 85.9, 'total_pymnt': 6281.4416701799, 
#               'revol_bal': 36170.0, 'annual_inc': 74000.0}
predicted_default_prob = predict_default_prob(est, row, cat_columns, cat_val_dict)



In [12]:
#################################################################

In [13]:
# Now moving to return rate estimation using second random forest regressor

In [14]:
# Estimating repay_retment rate of only_ret defaulted/charged off loans given loan characteristics
# Splitting the dataset
df_ret_v1 = df_rf.copy()

# reducing dataset to include only_ret those loans that have defaulted/charged off i.e. category_ret = 1
df_ret_v1 = df_ret_v1.loc[df_ret_v1['loan_status'].isin([1])]

# removing column loan_status because this has a value of 1 throughout now
df_ret_v1 = df_ret_v1.drop(['loan_status'], axis = 1)
y_ret = df_ret_v1['total_pymnt']
x_ret = df_ret_v1.drop(['total_pymnt'], axis = 1)

X_ret_train, X_ret_test, y_ret_train, y_ret_test = train_test_split(x_ret, y_ret, test_size=0.5, random_state=99)

In [20]:
# Fitting a random forest_ret
n_trees, maxf = 45, 14

est_ret = RandomForestRegressor(oob_score=True, n_estimators = n_trees, max_depth=20)
rf1_est = est_ret.fit(X_ret_train, y_ret_train)
print("OOB, accuracy_ret:", rf1_est.oob_score_)

train_acc_rf1_est = rf1_est.score(X_ret_train, y_ret_train)
print("The accuracy_ret on training set:",train_acc_rf1_est)

test_ret_acc_rf1_est = rf1_est.score(X_ret_test, y_ret_test)
print("The accuracy_ret on test_ret set:", test_ret_acc_rf1_est)

OOB, accuracy_ret: 0.5234142895871233
The accuracy_ret on training set: 0.8060828142855806
The accuracy_ret on test_ret set: 0.5346628788776852


In [21]:
# Predicting total payment for defaulted/charged off loans
print(est_ret.predict(X_ret_train))
# print(est.predict_proba(X_test))

# Assuming the target is (0,1) => (full_paid, default/charged-off), the pair of probabilities we get below
# 

[ 3447.36896399  7656.74272977 10841.28996375 ... 15533.30395259
  4422.39600451  4830.30601359]


In [22]:
# esimated_model is the model estimated using tarining data
# new_entry corresponds to a new loan that we are going to calculate default probability on
# cat_cols is the list of all categorical columns
# cat_val_dict is the dictionary of dictionaries that includes a categoerical variables and their 
# underlying entries from training data

def predict_default_pymnt(estimated_model, new_entry, cat_cols, cat_val_dict):
    #print(new_entry)
    # first for loop takes categorical columns of new_entry as input and updates their values 
    # according to the values in cat_val_dict
    # cat_val_dict includes all the parameter values from the training model
    for col in cat_cols:
        if(col != 'loan_status'):
            d = cat_val_dict[col]
            new_entry[col] = d[new_entry[col]]

    arr = np.array(list(new_entry.values())).reshape(1,len(new_entry))
    # now estimating the partial repayment for a new loan in case it defaults
    pymnt = estimated_model.predict(arr)
    return pymnt[0]  

In [23]:
# Let's test a new loan to see if our model correctly predicts default probability
data_ret = my_test_df.copy()
y_ret_var = data_ret['total_pymnt']
all_columns = []
all_columns = all_columns + cat_cts_columns
all_columns.remove('loan_status')
x_ret_var = data_ret[all_columns]
x_ret_var = x_ret_var.drop(['total_pymnt'], axis = 1)

row_ret = x_ret_var.iloc[0].to_dict()

# sample_row = {'sub_grade': 'E1', 'home_ownership': 'OWN', 'purpose': 'debt_consolidation', 
#               'grade': 'E', 'mort_acc': 2.0, 'emp_length': '6 years', 'addr_state': 'GA', 
#               'loan_amnt': 6000.0, 'int_rate': 22.9, 'dti': 32.77, 'inq_last_6mths': 0.0, 
#               'open_acc': 10.0, 'total_acc': 29.0, 'revol_util': 85.9, 'revol_bal': 36170.0, 
#               'annual_inc': 74000.0}
predicted_payment = predict_default_pymnt(est_ret, row_ret, cat_columns, cat_val_dict)

# adjust the 3 below as necessary
full_repayment = row_ret['loan_amnt']*(1.03)
full_repayment_prob = 1 - predicted_default_prob

In [29]:
print(row_ret)
print(x_ret_var.iloc[0])
print(y_ret_var.iloc[0])

{'sub_grade': 13, 'home_ownership': 2, 'purpose': 6, 'grade': 3, 'mort_acc': 2, 'emp_length': 7, 'addr_state': 46, 'loan_amnt': 6000.0, 'int_rate': 22.9, 'dti': 32.77, 'inq_last_6mths': 0.0, 'open_acc': 10.0, 'total_acc': 29.0, 'revol_util': 85.9, 'revol_bal': 36170.0, 'annual_inc': 74000.0}
sub_grade                         E1
home_ownership                   OWN
purpose           debt_consolidation
grade                              E
mort_acc                           2
emp_length                   6 years
addr_state                        GA
loan_amnt                       6000
int_rate                        22.9
dti                            32.77
inq_last_6mths                     0
open_acc                          10
total_acc                         29
revol_util                      85.9
revol_bal                      36170
annual_inc                     74000
Name: 43, dtype: object
6281.4416701799


In [30]:
# Let's develop a function to estimate return rate on a loan, train our model parameters, 
df_v3 = df_rf.copy()
expected_ret_amount = predicted_payment*predicted_default_prob + full_repayment*full_repayment_prob



expected_ret_rate = ((predicted_payment/row['loan_amnt'])*predicted_default_prob + \
                    (full_repayment/row['loan_amnt'])*full_repayment_prob)

print(expected_ret_rate)


0.048429415297625344
